
# Outline

This notebook provides the data analysis performed for a bike sharing case-study. The company has two types of riders: casual and annual members. The goal of the study was to identify how annual members and casual riders use the bikes differently in order to help the marketing team whose target is to convert casual riders into annual members in the next campaign. The structure of the notebook will be as follows:
* Importing
* Exploration
* Cleaning
* Preparation
* Analysis

# Importing
In this project, a MySQL DBMS was used. It should be noted that the statements mentioned in this section, were run from the MySQL command line, not from this notebook. In order to be able to import the data, the following was done:
* Enable the option to load the data from local files
* Create the database
* Create the tables
* Load data into the tables

In [ ]:
SET GLOBAL local_infile = 1;    
CREATE DATABASE bike_share_db;
use bike_share_db; 

In order to load the data, first a table for all the data was created, with an extra column month. Second, a temporary table was created with the original number of columns. 

In [ ]:
CREATE TABLE all_months (
ride_id VARCHAR (200), 
rideable_type VARCHAR (200), 
started_at DATETIME, 
ended_at DATETIME, 
start_station_name VARCHAR (200), 
start_station_id VARCHAR (200), 
end_station_name VARCHAR (200), 
end_station_id VARCHAR (200), 
start_lat VARCHAR (200), 
start_lng VARCHAR (200), 
end_lat VARCHAR (200), 
end_lng VARCHAR (200), 
member_casual VARCHAR (200),
month VARCHAR(200)
);

CREATE TEMPORARY TABLE temp_table (                  
    ride_id VARCHAR (200), 
    rideable_type VARCHAR (200), 
    started_at DATETIME, 
    ended_at DATETIME, 
    start_station_name VARCHAR (200), 
    start_station_id VARCHAR (200), 
    end_station_name VARCHAR (200), 
    end_station_id VARCHAR (200), 
    start_lat VARCHAR (200), 
    start_lng VARCHAR (200), 
    end_lat VARCHAR (200), 
    end_lng VARCHAR (200), 
    member_casual VARCHAR (200)
);

Then the following steps were repeated for each month:
* Load the data from the .csv file into the temporary table.
* Insert the data into the all_months table while inserting the month value.
* Delete the entries from the temporary table

In [ ]:

LOAD DATA LOCAL INFILE '202301-divvy-tripdata.csv' 
    INTO TABLE temp_table 
    FIELDS TERMINATED BY ',' 
    OPTIONALLY ENCLOSED BY '"' 
    LINES TERMINATED BY '\r\n' 
    IGNORE 1 ROWS;

INSERT INTO all_months SELECT *, 'Jan' AS month FROM temp_table;

TRUNCATE TABLE temp_table;

# Exploration:
First, we calculate the total number of rides per month, then the total and mean for the year 2023.

In [ ]:
SELECT  month, FORMAT(COUNT(*), 0) AS 'no of rides'
FROM    all_months
GROUP BY    month

UNION ALL

SELECT  'YEAR 2023', FORMAT(COUNT(*), 0) AS 'total_rides'
FROM    all_months

UNION ALL

SELECT  'Mean 2023', FORMAT(COUNT(*)/12, 0) 
FROM    all_months

Here I calculate the percentage of empty/null values. This is done by taking the SUM of the cases when the column value is empty, and then converting that to a percentage by dividing by COUNT. Finally the CAST function is used convert the percentage to a signed integer, making it easier to read. For the columns started_at and ended_at, where the datatype is DATETIME, the column value cannot be compared to an empty string, so instead, the function is NULL is used instead. For the columns where the result is less than 1% but not 0, the percentage was formatted to two decimal places. The result is saved to null_percentage.csv

In [ ]:
SELECT  month, 
        CAST(SUM( CASE WHEN ride_id             = '' THEN 1 ELSE 0 END)*100/COUNT(*) AS SIGNED) AS ride_id_null,
        CAST(SUM( CASE WHEN rideable_type       = '' THEN 1 ELSE 0 END)*100/COUNT(*) AS SIGNED) AS rideable_type_null,
        CAST(SUM( CASE WHEN started_at  is NULL THEN 1 ELSE 0 END)*100/COUNT(*)      AS SIGNED) AS started_at_null,
        CAST(SUM( CASE WHEN ended_at    is NULL THEN 1 ELSE 0 END)*100/COUNT(*)      AS SIGNED) AS ended_at_null,
        CAST(SUM( CASE WHEN start_station_name  = '' THEN 1 ELSE 0 END)*100/COUNT(*) AS SIGNED) AS start_station_name_null,
        CAST(SUM( CASE WHEN start_station_id    = '' THEN 1 ELSE 0 END)*100/COUNT(*) AS SIGNED) AS start_station_id_null,
        CAST(SUM( CASE WHEN end_station_name    = '' THEN 1 ELSE 0 END)*100/COUNT(*) AS SIGNED) AS end_station_name_null,
        CAST(SUM( CASE WHEN end_station_id      = '' THEN 1 ELSE 0 END)*100/COUNT(*) AS SIGNED) AS end_station_id_null,
        CAST(SUM( CASE WHEN start_lat           = '' THEN 1 ELSE 0 END)*100/COUNT(*) AS SIGNED) AS start_lat_null,
        CAST(SUM( CASE WHEN start_lng           = '' THEN 1 ELSE 0 END)*100/COUNT(*) AS SIGNED) AS start_lng_null,
        FORMAT(SUM( CASE WHEN end_lat           = '' THEN 1 ELSE 0 END)*100/COUNT(*),2)  AS end_lat_null,
        FORMAT(SUM( CASE WHEN end_lng           = '' THEN 1 ELSE 0 END)*100/COUNT(*),2)  AS end_lng_null,
        CAST(SUM( CASE WHEN member_casual       = '' THEN 1 ELSE 0 END)*100/COUNT(*) AS SIGNED) AS member_casual_null
FROM    all_months
GROUP BY    month

Here i find the number of distinct values for each column.

In [ ]:
SELECT  month,
        COUNT(DISTINCT(ride_id))                AS ride_id_uni,
        COUNT(DISTINCT(rideable_type))          AS rideable_type_uni,
        COUNT(DISTINCT(started_at))             AS started_at_uni,
        COUNT(DISTINCT(ended_at))               AS ended_at_uni,
        COUNT(DISTINCT(start_station_name))     AS start_station_name_uni,
        COUNT(DISTINCT(start_station_id))       AS start_station_id_uni,
        COUNT(DISTINCT(end_station_name))       AS end_station_name_uni,
        COUNT(DISTINCT(end_station_id))         AS end_station_id_uni,
        COUNT(DISTINCT(start_lat))              AS start_lat_uni,
        COUNT(DISTINCT(start_lng))              AS start_lng_uni,
        COUNT(DISTINCT(end_lat))                AS end_lat_uni,
        COUNT(DISTINCT(end_lng))                AS end_lng_uni,
        COUNT(DISTINCT(member_casual))          AS member_casual_uni
FROM    all_months
GROUP BY month

# Cleaning
After seeing that the ride length can be calculated from the columns started_at and ended_at and that the columns related to ride location contains nulls and discrepancies, these columns will be dropped. But in order to leave the database unaltered, here i select entries where ended_at time is after started_at time only, and create a new table with the columns that will be needed, which are:
* rideable_type
* a new column ride_length = ended_at - started_at
* a new column day_of_week extracted from the column started_at
* member_casual
* month

In [ ]:
CREATE TABLE    all_months_cleaned AS
SELECT          
                rideable_type, 
                TIMESTAMPDIFF(SECOND, started_at, ended_at) AS ride_length, 
                DAYOFWEEK(started_at) AS day_of_week, 
                member_casual, 
                month
FROM            all_months
WHERE           ended_at >= started_at

# Analysis:
Here i find the number of rides per rider type: member/casual per month and for the year 2023.

In [ ]:
select  month as 'Month',
        FORMAT(SUM(CASE WHEN member_casual = 'casual' THEN 1 ELSE 0 END), 0) AS 'Casual',
        FORMAT(SUM(CASE WHEN member_casual = 'member' THEN 1 ELSE 0 END), 0) AS 'Member'
FROM    all_months_cleaned
GROUP BY month

union all 

select  'Year 2023',
        FORMAT(SUM(CASE WHEN member_casual = 'casual' THEN 1 ELSE 0 END), 0) AS 'Casual',
        FORMAT(SUM(CASE WHEN member_casual = 'member' THEN 1 ELSE 0 END), 0) AS 'Member'
from    all_months_cleaned


Next, i get the mean and maximum ride length per rider type. The method SEC_TO_TIME converts the average ride_length from seconds to HH:MM:SS, the method DATE_FORMAT then allows us to format the date since the average method returns too many decimals. For the maximum, the value is converted to days since these rides are much too long to be displayed in HH:MM:SS. 

In [ ]:
select  month as 'Month',
        DATE_FORMAT(SEC_TO_TIME(AVG(CASE WHEN member_casual = 'casual' THEN ride_length END)), '%H:%i:%s') AS 'Mean_Casual',
        DATE_FORMAT(SEC_TO_TIME(AVG(CASE WHEN member_casual = 'member' THEN ride_length END)), '%H:%i:%s') AS 'Mean_Member',
        CONCAT(FLOOR(MAX(CASE WHEN member_casual = 'casual' THEN ride_length END) / (24*3600)), ' days') AS 'Max_Casual',
        CONCAT(FLOOR(MAX(CASE WHEN member_casual = 'member' THEN ride_length END) / (24*3600)), ' days') AS 'Max_Member'
FROM    all_months_cleaned
GROUP BY month

union all 

select  'Year 2023',
        DATE_FORMAT(SEC_TO_TIME(AVG(CASE WHEN member_casual = 'casual' THEN ride_length END)), '%H:%i:%s') AS 'Mean_Casual',
        DATE_FORMAT(SEC_TO_TIME(AVG(CASE WHEN member_casual = 'member' THEN ride_length END)), '%H:%i:%s') AS 'Mean_Member',
        CONCAT(FLOOR(MAX(CASE WHEN member_casual = 'casual' THEN ride_length END) / (24*3600)), ' days') AS 'Max_Casual',
        CONCAT(FLOOR(MAX(CASE WHEN member_casual = 'member' THEN ride_length END) / (24*3600)), ' days') AS 'Max_Member'
from    all_months_cleaned

Here i find the mean ride length per rider type after grouping the data by day of the week

In [ ]:
SELECT      day_of_week,
            DATE_FORMAT(SEC_TO_TIME(AVG(CASE WHEN member_casual = 'casual' THEN ride_length END)), '%H:%i:%s')  AS 'Mean_Casual',
            DATE_FORMAT(SEC_TO_TIME(AVG(CASE WHEN member_casual = 'member' THEN ride_length END)), '%H:%i:%s')  AS 'Mean_Member'
FROM        all_months_cleaned
GROUP BY    day_of_week
ORDER BY    day_of_week

Here i find the number of bike rides per rider type per type of bike

In [ ]:
SELECT      rideable_type,    
            FORMAT(SUM(CASE WHEN member_casual='casual' THEN 1 ELSE 0 END), 0) AS 'Casual',
            FORMAT(SUM(CASE WHEN member_casual='member' THEN 1 ELSE 0 END), 0) AS 'Member'
FROM        all_months_cleaned
GROUP BY    rideable_type